#### Title:          08_framework_vs_vicon.ipynb
#### Description:    Compare a framework with Vicon
#### Authors:        Fabian Kahl and Maximilian Kapsecker

### 1. General Functionality and Definitions

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from scipy import signal
from statsmodels.tsa.stattools import grangercausalitytests
import statsmodels.api as sm
import pingouin as pg
import warnings
from utils import *

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.precision', 2)
path_to_complete_csv = '../data/csvs_complete/'

# Frameworks:
# 'AlphaPose', 'Detectron2', 'MediaPipe', 'MeTRAbs', 'MHFormer',
# 'MMPose', 'MoveNet', 'OpenPifPaf', 'OpenPifPaf-vita', 'OpenPose', 'PoseFormerV2',
# 'rtmlib', 'StridedTransformer-Pose3D', 'ultralytics', 'ViTPose', 'YOLOv7'
framework = 'MoveNet'
path_best_shifts = '../data/best_shifts/'

In [ ]:
# Column name mapping
mapping = {
    'leftShoulder': 'LSJC',
    'leftElbow': 'LEJC',
    'leftHand': 'LWJC',
    'leftHip': 'LHJC',
    'leftKnee': 'LKJC',
    'leftFoot': 'LAJC',
    'rightShoulder': 'RSJC',
    'rightElbow': 'REJC',
    'rightHand': 'RWJC',
    'rightHip': 'RHJC',
    'rightKnee': 'RKJC',
    'rightFoot': 'RAJC'
}

# Parameters
exercises = ['Front lunge', 'Jumping jacks', 'Lateral arm raise',
             'Leg extension crunch', 'Reverse fly', 'Side squat',
             'Single leg deadlift', 'Squat']

views = ['Frontal', 'Side']

# parent - angle - child
angles = [['leftHip', 'leftShoulder', 'leftElbow'],
          ['leftShoulder', 'leftElbow', 'leftHand'],
          ['rightHip', 'rightShoulder', 'rightElbow'],
          ['rightShoulder', 'rightElbow', 'rightHand'],
          ['leftShoulder', 'leftHip', 'leftKnee'],
          ['leftHip', 'leftKnee', 'leftFoot'],
          ['rightShoulder', 'rightHip', 'rightKnee'],
          ['rightHip', 'rightKnee', 'rightFoot'],
         ]

relevant_angles = {
    'Squat' : 
    [
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ],
    'Front lunge' : 
    [
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ],
    'Single leg deadlift' : [
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ],
    'Lateral arm raise' : 
    [
        ['leftHip', 'leftShoulder', 'leftElbow'], 
        ['rightHip', 'rightShoulder', 'rightElbow'], 
        ['leftShoulder', 'leftElbow', 'leftHand'], 
        ['rightShoulder', 'rightElbow', 'rightHand']
    ],
    'Reverse fly' : 
    [
        ['leftHip', 'leftShoulder', 'leftElbow'], 
        ['rightHip', 'rightShoulder', 'rightElbow'], 
        ['leftShoulder', 'leftElbow', 'leftHand'], 
        ['rightShoulder', 'rightElbow', 'rightHand']
    ],
    'Jumping jacks' : 
    [
        ['leftHip', 'leftShoulder', 'leftElbow'], 
        ['rightHip', 'rightShoulder', 'rightElbow'], 
        ['leftShoulder', 'leftElbow', 'leftHand'], 
        ['rightShoulder', 'rightElbow', 'rightHand'], 
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ],
    'Side squat' : [
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ],
    'Leg extension crunch' : 
    [
        ['leftShoulder', 'leftHip', 'leftKnee'], 
        ['rightShoulder', 'rightHip', 'rightKnee'], 
        ['leftHip', 'leftKnee', 'leftFoot'], 
        ['rightHip', 'rightKnee', 'rightFoot']
    ]
}

angles_benchmark = pd.DataFrame(index=exercises, columns=views)
angles_benchmark.loc['Front lunge'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                       ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Jumping jacks'] = [['leftHip', 'leftShoulder', 'leftElbow'],
                                         ['leftHip', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Lateral arm raise'] = [['leftHip', 'leftShoulder', 'leftElbow'],
                                            ['leftHip', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Leg extension crunch'] = [['rightShoulder', 'rightHip', 'rightKnee'],
                                                ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Reverse fly'] = [['rightShoulder', 'leftShoulder', 'leftElbow'],
                                      ['rightShoulder', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Side squat'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                     ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Single leg deadlift'] = [['leftShoulder', 'leftHip', 'leftKnee'],
                                              ['leftShoulder', 'leftHip', 'leftKnee']]
angles_benchmark.loc['Squat'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                ['rightHip', 'rightKnee', 'rightFoot']]

multi_index = pd.MultiIndex.from_product([exercises, views], names=['Exercise', 'View'])
data = [
    ['Not Visible', 'No Action', 'Not Visible', 'No Action', 'Side View', 'Side View', 'Side View', 'Side View'],
    ['No Action', 'No Action', 'No Action', 'No Action', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View'],
    ['Oblique View', 'Oblique View', 'Side View', 'Side View', 'Oblique View', 'Oblique View', 'Frontal View', 'Frontal View'],
    ['Oblique View', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View'],
    ['No Action', 'No Action', 'Side View', 'Side View', 'No Action', 'No Action', 'No Action', 'No Action'],
    ['No Action', 'No Action', 'Oblique View', 'Oblique View', 'No Action', 'No Action', 'No Action', 'No Action'],
    ['Not Visible', 'Oblique View', 'Not Visible', 'Oblique View', 'Side View', 'Side View', 'Side View', 'Side View'],
    ['Not Visible', 'Oblique View', 'Not Visible', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View'],
    ['No Action', 'No Action', 'Frontal View', 'Frontal View', 'No Action', 'No Action', 'No Action', 'No Action'],
    ['No Action', 'No Action', 'Oblique View', 'Oblique View', 'No Action', 'No Action', 'No Action', 'No Action'],
    ['No Action', 'No Action', 'No Action', 'No Action', 'Oblique View', 'Oblique View', 'No Action', 'Oblique View'],
    ['No Action', 'No Action', 'No Action', 'No Action', 'Oblique View', 'Oblique View', 'No Action', 'Side View'],
    ['No Action', 'No Action', 'Not Visible', 'Side View', 'Side View', 'No Action', 'Side View', 'No Action'],
    ['No Action', 'No Action', 'Oblique View', 'Oblique View', 'Oblique View', 'No Action', 'Oblique View', 'No Action'],
    ['No Action', 'No Action', 'No Action', 'No Action', 'Frontal View', 'Frontal View', 'Frontal View', 'Frontal View'],
    ['No Action', 'No Action', 'No Action', 'No Action', 'Oblique View', 'Oblique View', 'Oblique View', 'Oblique View']
]
angle_order = ['leftElbow', 'rightElbow', 'leftShoulder', 'rightShoulder', 'leftHip', 'rightHip', 'leftKnee', 'rightKnee']
angle_categorization = pd.DataFrame(index=multi_index, columns=angle_order, data=data)

In [ ]:
angle_categorization

### 2. Loading Data

In [ ]:
df = pd.read_csv(path_to_complete_csv+framework+'_vs_vicon_complete.csv')

In [ ]:
df.head()

In [ ]:
mean_sr = []
for k in df.subject.unique():
    for i in df.exerciseName.unique():
        for j in df.ipad_view.unique():
            mean_time = np.mean(
                    df[(df.subject == k) & (df.exerciseName == i) & (df.ipad_view == j)].timestamp.diff()
                )
            # Hence not all combinations are available, skip not existing ones
            if mean_time is not np.nan:
                mean_sr.append(mean_time)

print('Average Frequency for '+framework+':', 1 / np.mean(mean_sr), 'Hz')

### 3. Analyse Data

Iterating through all combinations and calculating for each combination the mean and mutual information. The results are compiled in a common dataframe.

In [ ]:
result = pd.DataFrame()
save_plot = False
df_best_shifts = pd.read_csv(path_best_shifts+'best_shifts.csv')

for subject in [1, 2, 3, 4, 6, 7, 8, 9, 10]:
    for exercise in exercises:
        for view in views:
            try:
                best_shift = int(df_best_shifts.loc[(df_best_shifts['Subject'] == subject) &
                                (df_best_shifts['Exercise'] == exercise) &
                                (df_best_shifts['View'] == view),
                                'best_shift_median'].iloc[0])
            except:
                print('No best_shift for combination: ', str(subject), exercise, view)
                break
            for angle in angles:
                try:
                    angles_ar, angles_vi, config = compute_angles(
                        df,
                        angle,
                        mapping,
                        ipad_view=view,
                        subject=subject,
                        exercise=exercise,
                    )

                    angles_ar, angles_vi = shift_timeseries(angles_ar, angles_vi, best_shift)
                    angles_ar, angles_vi = cut_timeseries(angles_ar, angles_vi)
                    shift, best_error = baseline_drift(angles_ar, angles_vi)

                    mean_error = np.array(angles_ar) - np.array(angles_vi)
                    diff_abs = np.abs(mean_error)
                    cc = signal.correlate(np.array(angles_ar), np.array(angles_vi))

                    # Enable if you are not running on Apple Silicon
                    mInformation = 0 #pyi.mutual_info(np.round(angles_ar), np.round(angles_vi))

                    pearson, pvalue, zTrafo, lci, uci = pearsonr_ci(angles_ar, angles_vi)
                    spearman, sp_pvalue, sp_zTrafo, sp_lci, sp_uci = spearmanr_ci(angles_ar, angles_vi)

                    with warnings.catch_warnings():
                        # Ignore all caught warnings including FutureWarning from verbose
                        warnings.filterwarnings("ignore")

                        gc_res = grangercausalitytests(
                            pd.DataFrame([angles_ar, angles_vi]).transpose(),
                            [60],
                            verbose=False # Depricated
                        )

                    centerMoved = exercise in ['Squat', 'Front lunge', 'Jumping jacks', 'Side squat']
                    lowerBody = angle[1] in ['leftHip', 'rightHip', 'leftKnee', 'rightKnee']
                    leftBodySide = angle[1] in ['leftHip', 'leftKnee', 'leftElbow', 'leftShoulder']
                    sex = subject in [3, 4, 7, 10]
                    df_angles_ar = pd.DataFrame(data=angles_ar, columns=['Angle'])
                    df_angles_ar['Framework'] = framework
                    df_angles_ar['Frame'] = range(1, len(df_angles_ar) + 1)
                    df_angles_vi = pd.DataFrame(data=angles_vi, columns=['Angle'])
                    df_angles_vi['Framework'] = 'VICON'
                    df_angles_vi['Frame'] = range(1, len(df_angles_vi) + 1)
                    df_angles = pd.concat([df_angles_ar, df_angles_vi], ignore_index=True)

                    d = {
                        'Angle': [angle[1]],
                        'Angle Categorization': [angle_categorization.loc[(exercise, view), angle[1]]],
                        'View': [view],
                        'Subject': [subject],
                        'Exercise': [exercise],
                        'Relevant': angle in relevant_angles[exercise],
                        'MeanAbsoluteError': np.mean(diff_abs),
                        'MeanError': np.mean(mean_error),
                        'LogTrafoMAE': np.log(np.mean(diff_abs)),
                        'TotalAbsoluteError': np.sum(diff_abs),
                        'Overestimates': len(np.where(mean_error > 0.0)[0]) / len(diff_abs),
                        'Underestimates': len(np.where(mean_error < 0.0)[0]) / len(diff_abs),
                        'OverestimatesWeighted': np.sum(np.where(mean_error > 0.0)[0]) / len(diff_abs),
                        'UnderestimatesWeighted': np.sum(np.where(mean_error < 0.0)[0]) / len(diff_abs),
                        'SampleSize': len(diff_abs),
                        'MutualInformation': mInformation,
                        'PearsonCorrelation': pearson,
                        'MAE99q': np.quantile(diff_abs, 0.99),
                        'ME99q': np.quantile(mean_error, 0.99),
                        'PearsonPValue': pvalue,
                        'PearsonzTrafo': zTrafo,
                        'PearsonLowerCI': lci,
                        'PearsonUpperCI': uci,
                        'SpearmanCorrelation': spearman,
                        'SpearmanPValue': sp_pvalue,
                        'GrangertestFValue': gc_res[60][0]['params_ftest'][0],
                        'GrangertestPValue': gc_res[60][0]['params_ftest'][1],
                        'CenterMoved': centerMoved,
                        'LowerBody': lowerBody,
                        'LeftBodySide': leftBodySide,
                        'Sex': sex,
                        'BaseLineDrift': shift,
                        'BestError': best_error,
                        'Valid': True,
                        'ICC2': pg.intraclass_corr(data=df_angles, targets='Frame', raters='Framework',
                                 ratings='Angle', nan_policy='omit')['ICC'][1],
                    }
                except Exception as e:
                    print(e, 'Angle:', angle[1], 'View:', view, 'Subject:', subject, 'Exercise:', exercise)
                    d = {
                        'Angle': [angle[1]],
                        'Angle Categorization': [angle_categorization.loc[(exercise, view), angle[1]]],
                        'View': [view],
                        'Subject': [subject],
                        'Exercise': [exercise],
                        'Relevant': 0,
                        'MeanAbsoluteError': 0,
                        'MeanError': 0,
                        'LogTrafoMAE': 0,
                        'TotalAbsoluteError': 0,
                        'Overestimates': 0,
                        'Underestimates': 0,
                        'OverestimatesWeighted': 0,
                        'UnderestimatesWeighted': 0,
                        'SampleSize': 0,
                        'MutualInformation': 0,
                        'PearsonCorrelation': 0,
                        'MAE99q': 0,
                        'ME99q': 0,
                        'PearsonPValue': 0,
                        'PearsonzTrafo': 0,
                        'PearsonLowerCI': 0,
                        'PearsonUpperCI': 0,
                        'SpearmanCorrelation': 0,
                        'SpearmanPValue': 0,
                        'GrangertestFValue': 0,
                        'GrangertestPValue': 0,
                        'CenterMoved': 0,
                        'LeftBodySide': 0,
                        'LowerBody': 0,
                        'Sex': 0,
                        'BaseLineDrift': 0,
                        'BestError': 0,
                        'Valid': False,
                        'ICC2': 0,
                    }

                result = pd.concat([result, pd.DataFrame(data=d)], ignore_index=True)

                if save_plot:
                    if d['Valid'] == True:
                        fig = plt.figure(figsize=(20,8))
                        plt.plot(angles_ar, c='b', label=framework)

                        plt.plot(angles_vi, c='g', label='Vicon')
                        fig.suptitle(config, fontsize=20)
                        plt.plot(diff_abs, c='r', label='Difference')
                        plt.legend(loc='upper left')
                        plt.savefig('./media/' + angle[1] + '_' + view + '_subject' + str(subject) + '_' + exercise)
                        plt.close()

### 4. Analysis of the results

In [ ]:
column_order = [
    'leftElbow',
    'rightElbow',
    'leftShoulder',
    'rightShoulder',
    'leftHip',
    'rightHip',
    'leftKnee',
    'rightKnee',
]

In [ ]:
# Filter out the samples for which the calculation was successful
result = result[result.Valid == True].copy()
# Filter out View No Action
result = result[result['Angle Categorization'] != 'Not Visible'].copy()
n = len(result)
result.index = range(0, n)
print('Number of ', framework, '-Vicon time series comparisons:', n)

In [ ]:
result.loc[:, 'LogMAE'] = np.log(result['MeanAbsoluteError'])

# Weighted aggregation wrt length of the time series
wm = lambda x: np.average(x, weights=result.loc[x.index, "SampleSize"])
wmstd = lambda x: weighted_avg_and_std(x, weights=result.loc[x.index, "SampleSize"])

#### 4.1 ANOVA

In [ ]:
anova_result = pd.DataFrame()
for k in ['LogMAE']:
    for i in ['Angle', 'Exercise', 'Subject', 'Angle Categorization']:
        temp = result.welch_anova(dv=k, between=i)
        temp['response'] = k
        temp['categorical'] = i
        anova_result = pd.concat([anova_result, temp], ignore_index=True)
        
anova_result.index = range(len(anova_result))

In [ ]:
anova_result

In [ ]:
anova_result_side = pd.DataFrame()
for k in ['LogMAE']:
    for i in ['Angle', 'Exercise', 'Subject']:
        temp = result[result['Angle Categorization'] == 'Side View'].welch_anova(dv=k, between=i)
        temp['response'] = k
        temp['categorical'] = i
        anova_result_side = pd.concat([anova_result_side, temp], ignore_index=True)
        
anova_result_side.index = range(len(anova_result_side))

In [ ]:
anova_result_side

#### 4.2 t-test

In [ ]:
result['View_Number'] = result['View'].replace('Frontal', 0).replace('Side', 1)

In [ ]:
ttest_result = pd.DataFrame()
for k in ['LogMAE']:
    for i in ['View_Number', 'CenterMoved', 'Sex']:
        print(k, i)
        x = result[result[i] == 0][k]
        y = result[result[i] == 1][k]
        temp = pg.ttest(x, y, correction=True)
        temp['response'] = k
        temp['categorical'] = i
        ttest_result = pd.concat([ttest_result, temp], ignore_index=True)
ttest_result.index = range(len(ttest_result))

In [ ]:
ttest_result

In [ ]:
ttest_result_side = pd.DataFrame()
for k in ['LogMAE']:
    for i in ['View_Number', 'CenterMoved', 'Sex']:
        print(k, i)
        result_temp = result[result['Angle Categorization'] == 'Side View']
        x = result_temp[result_temp[i] == 0][k]
        y = result_temp[result_temp[i] == 1][k]
        temp = pg.ttest(x, y, correction=True)
        temp['response'] = k
        temp['categorical'] = i
        ttest_result_side = pd.concat([ttest_result_side, temp], ignore_index=True)
ttest_result_side.index = range(len(ttest_result_side))

In [ ]:
ttest_result_side

#### 4.3 Mean Absolute Error

In [ ]:
print("Overall wMAE:", result.MeanAbsoluteError.agg(wmstd))
print("Side wMAE:", result[result['Angle Categorization'] == 'Side View'].MeanAbsoluteError.agg(wmstd))

In [ ]:
agg_weighted_mae_w_std = pd.pivot_table(
    result,
    values='MeanAbsoluteError',
    index=['Angle Categorization'],
    columns=['Angle'],
    aggfunc=wmstd
)[column_order]
agg_weighted_mae = pd.pivot_table(
    result,
    values='MeanAbsoluteError',
    index=['Angle Categorization'],
    columns=['Angle'],
    aggfunc=wm
)[column_order]

In [ ]:
styler = agg_weighted_mae.style
styler.background_gradient(axis=None, cmap='RdYlGn_r', vmin=0.0, vmax= 45.0)
styler

In [ ]:
agg_weighted_mae_w_std

In [ ]:
weighted_MAEs_path = '../paper/weightedMAEs.xlsx'

if not os.path.exists(os.path.dirname(weighted_MAEs_path)):
    os.makedirs(os.path.dirname(weighted_MAEs_path))

if not os.path.isfile(weighted_MAEs_path):
    pd.DataFrame().to_excel(weighted_MAEs_path, index=False)

with pd.ExcelWriter(weighted_MAEs_path, engine='openpyxl', mode='a') as writer:
    styler.to_excel(writer, sheet_name=framework+'_int')
    agg_weighted_mae_w_std.to_excel(writer, sheet_name=framework+'_std', engine='openpyxl')

#### 4.4 ICCs

In [ ]:
print("Overall weighted ICC2:", result.ICC2.agg(wmstd))
print("Side weighted ICC2:", result[result['Angle Categorization'] == 'Side View'].ICC2.agg(wmstd))

In [ ]:
icc2_mae = pd.pivot_table(
    result,
    values='ICC2',
    index=['Angle Categorization'],
    columns=['Angle'],
    aggfunc=wm
)[column_order]
icc2_mae_w_std = pd.pivot_table(
    result,
    values='ICC2',
    index=['Angle Categorization'],
    columns=['Angle'],
    aggfunc=wmstd
)[column_order]

In [ ]:
styler = icc2_mae.style
styler.background_gradient(axis=None, cmap='RdYlGn', vmin=0.0, vmax= 1.0)
styler

In [ ]:
icc2_mae_w_std

In [ ]:
weighted_ICCs_path = '../paper/weightedICCs.xlsx'

if not os.path.exists(os.path.dirname(weighted_ICCs_path)):
    os.makedirs(os.path.dirname(weighted_ICCs_path))

if not os.path.isfile(weighted_ICCs_path):
    pd.DataFrame().to_excel(weighted_ICCs_path, index=False)
    
with pd.ExcelWriter(weighted_ICCs_path, engine='openpyxl', mode='a') as writer:
    styler.to_excel(writer, sheet_name=framework+'_int')
    icc2_mae_w_std.to_excel(writer, sheet_name=framework+'_std', engine='openpyxl')